### Model Structure
* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

In [10]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

In [11]:
input = Input(shape=(150, 150, 3))
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input)
layer2 = MaxPooling2D(pool_size=(2, 2))(layer1)
layer3 = Flatten()(layer2)
layer4 = Dense(64, activation='relu')(layer3)
output = Dense(1, activation='sigmoid')(layer4)

model = Model(inputs=input, outputs=output)

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

**A: for binary classification problems, binary crossentropy should be the best loss function.**

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

**A: 11215873**

### Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validation 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [18]:
gen = ImageDataGenerator(rescale=1./255)

train_generator = gen.flow_from_directory(
    'archive/train',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    shuffle=True
)

validation_generator = gen.flow_from_directory(
    'archive/test',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    shuffle=True
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [19]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
80/80 [==============================] - 21s 250ms/step - loss: 0.6249 - accuracy: 0.6330 - val_loss: 0.5075 - val_accuracy: 0.7614
Epoch 2/10
80/80 [==============================] - 12s 149ms/step - loss: 0.4616 - accuracy: 0.7974 - val_loss: 0.4181 - val_accuracy: 0.8249
Epoch 3/10
80/80 [==============================] - 12s 153ms/step - loss: 0.3890 - accuracy: 0.8356 - val_loss: 0.5077 - val_accuracy: 0.7335
Epoch 4/10
80/80 [==============================] - 12s 152ms/step - loss: 0.3297 - accuracy: 0.8714 - val_loss: 0.3204 - val_accuracy: 0.8629
Epoch 5/10
80/80 [==============================] - 12s 156ms/step - loss: 0.2872 - accuracy: 0.8890 - val_loss: 0.3463 - val_accuracy: 0.8350
Epoch 6/10
80/80 [==============================] - 13s 158ms/step - loss: 0.2673 - accuracy: 0.9040 - val_loss: 0.2943 - val_accuracy: 0.8858
Epoch 7/10
80/80 [==============================] - 12s 150ms/step - loss: 0.2185 - accuracy: 0.9184 - val_loss: 0.2762 - val_accuracy: 0.8807

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

In [29]:
history.history

{'loss': [0.6249135136604309,
  0.46162909269332886,
  0.38897058367729187,
  0.3297272324562073,
  0.2872343957424164,
  0.2672862112522125,
  0.21846544742584229,
  0.21099267899990082,
  0.18368424475193024,
  0.15593062341213226],
 'accuracy': [0.6329987645149231,
  0.797365128993988,
  0.8356336355209351,
  0.8713927268981934,
  0.8889585733413696,
  0.904015064239502,
  0.9184441566467285,
  0.9196988940238953,
  0.9378920793533325,
  0.9535759091377258],
 'val_loss': [0.507524847984314,
  0.41813090443611145,
  0.5077062845230103,
  0.32040873169898987,
  0.34630948305130005,
  0.29425951838493347,
  0.2762324810028076,
  0.3284505605697632,
  0.2814604938030243,
  0.2678177058696747],
 'val_accuracy': [0.7614213228225708,
  0.8248730897903442,
  0.7335025668144226,
  0.8629441857337952,
  0.8350253701210022,
  0.8857868313789368,
  0.8807106614112854,
  0.8654822111129761,
  0.8680202960968018,
  0.8908629417419434]}

In [26]:
from statistics import median

In [22]:
median(history.history['accuracy'])

0.8964868187904358


### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

In [24]:
from statistics import stdev

In [25]:
stdev(history.history['loss'])

0.1448835113570336

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [31]:
gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

train_generator = gen.flow_from_directory(
    'archive/train',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    shuffle=True
)

validation_generator = gen.flow_from_directory(
    'archive/test',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    shuffle=True
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [33]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
80/80 [==============================] - 21s 262ms/step - loss: 0.4120 - accuracy: 0.8093 - val_loss: 0.4023 - val_accuracy: 0.8147
Epoch 2/10
80/80 [==============================] - 20s 253ms/step - loss: 0.3926 - accuracy: 0.8199 - val_loss: 0.4122 - val_accuracy: 0.8071
Epoch 3/10
80/80 [==============================] - 20s 250ms/step - loss: 0.3710 - accuracy: 0.8300 - val_loss: 0.3533 - val_accuracy: 0.8477
Epoch 4/10
80/80 [==============================] - 20s 252ms/step - loss: 0.3439 - accuracy: 0.8614 - val_loss: 0.3306 - val_accuracy: 0.8680
Epoch 5/10
80/80 [==============================] - 20s 255ms/step - loss: 0.3622 - accuracy: 0.8457 - val_loss: 0.3906 - val_accuracy: 0.8503
Epoch 6/10
80/80 [==============================] - 20s 253ms/step - loss: 0.3586 - accuracy: 0.8425 - val_loss: 0.3751 - val_accuracy: 0.8553
Epoch 7/10
80/80 [==============================] - 20s 256ms/step - loss: 0.3509 - accuracy: 0.8394 - val_loss: 0.3280 - val_accuracy: 0.8629

In [27]:
from statistics import mean

In [34]:
mean(history.history['val_loss'])

0.3626246780157089

### Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [35]:
mean(history.history['val_accuracy'][5:])

0.8573604106903077

In [43]:
## extra exploration

In [45]:
from tensorflow.keras.preprocessing.image import load_img

In [53]:
img = load_img('./archive/test/dragon/1e07914f-530e-4087-a377-cded306c360c.jpg', target_size=(150,150))
X = np.array(img)
X = np.array([X])
X.shape

(1, 150, 150, 3)

In [50]:
from tensorflow.keras.applications.xception import preprocess_input

In [54]:
preprocess_input(X)

array([[[[-0.18431371,  0.12941182,  0.14509809],
         [-0.15294117,  0.16078436,  0.17647064],
         [-0.09803921,  0.21568632,  0.2313726 ],
         ...,
         [-0.09019607,  0.2313726 ,  0.26274514],
         [-0.11372548,  0.20784318,  0.23921573],
         [-0.11372548,  0.20784318,  0.23921573]],

        [[-0.16862744,  0.14509809,  0.16078436],
         [-0.14509803,  0.1686275 ,  0.18431377],
         [-0.12941176,  0.18431377,  0.20000005],
         ...,
         [-0.11372548,  0.20784318,  0.23921573],
         [-0.02745098,  0.2941177 ,  0.32549024],
         [-0.12156862,  0.20000005,  0.2313726 ]],

        [[-0.19215685,  0.12156868,  0.13725495],
         [-0.10588235,  0.20784318,  0.22352946],
         [-0.0745098 ,  0.23921573,  0.254902  ],
         ...,
         [-0.08235294,  0.23921573,  0.27058828],
         [-0.09019607,  0.2313726 ,  0.26274514],
         [-0.1607843 ,  0.16078436,  0.19215691]],

        ...,

        [[-0.12156862,  0.22352946,  0

In [56]:
pred = model.predict(X)
pred[0]

1/1 [==============================] - 0s 28ms/step


array([1.], dtype=float32)

In [57]:
train_generator.class_indices

{'dino': 0, 'dragon': 1}

yay